 ## 整合了论文的代码，看一份notebook就足够了 
 
Author：Qiuyuanchen chenqiuyuan@zju.edu.cn

This structure the content of the paper and the corresponding codes.

# 准备阶段
## 设置路径

In [1]:
# 设置路径用的，不要运行多次
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
import os
try:
    os.chdir(os.path.join(os.getcwd(), '..'))
    print(os.getcwd())
except:
    pass
# os.startfile(os.getcwd())

C:\Users\Fallaway\OneDrive\UI_paper\python


 ## 引入数据路径，配置文件为path.config
 Get the needed path

In [1]:
import os
import numpy
import pandas as pd
import configparser
import sys
import configparser
def path(name='free', show=False):
    config = configparser.ConfigParser()
    config.read('C:\\Users\\Fallaway\\OneDrive\\UI_paper\\python\\data_process\\path.config')
    if show==True:
        return config.items('path')
    root = config.get('path', 'root')
    data_path = config.get('path', name)
    return os.path.join(root, data_path)
print(path(show=True))

[('root', 'C:/Users/Fallaway/OneDrive - business/UI_Review/data/'), ('meta', 'UI_Data/Apps_Data.csv'), ('meta_free_ui', 'UI_Data/Meta_Free_UI.csv'), ('meta_non_free_ui', 'UI_Data/Meta_Non_Free_UI.csv'), ('free', 'UI_Data/Free_Apps_Reviews.csv.gz'), ('non_free', 'UI_Data/Non_Free_Apps_Reviews.csv.gz'), ('free_ui', 'UI_Data/Free_UI.csv'), ('non_free_ui', 'UI_Data/Non_Free_UI.csv'), ('manual', 'manual/'), ('reply_data', 'C:/Users/Fallaway/OneDrive/Reviews_Revisions.csv.gz')]


 ## 加载原始数据
Load the raw data and filtered data.数据量超大，谨慎运行:可以使用nrows限制读取行数，usecols指定列名

In [3]:
# Load the raw data
# meta_data = pd.read_csv(path('meta'))
# free = pd.read_csv(path('free'), iterator=True)
# non_free = pd.read_csv(path('non_free'))
# pd.read_csv(path('free'), nrows=1000, usecols=['REVIEW_TEXT'])
# free_ui = pd.read_csv(path('free_UI'))
# non_free_ui = pd.read_csv(path('non_free_UI'))
# free = pd.read_csv(path('free'), usecols=['REVIEW_TEXT'])
# non_free = pd.read_csv(path('non_free'), usecols=['REVIEW_TEXT'])

# 数据处理与描述

## 检查app_review 与 app_data 中的app是否一致
分别从free的检查和non-free的检查

In [ ]:
# Data check
# identify the consistency of review and meta_data
meta_app = pd.Series(meta_data['PACKAGE_NAME'].unique())
free_app = pd.Series(free_ui['PACKAGE_NAME'].unique())
non_free_app = pd.Series(non_free_ui['PACKAGE_NAME'].unique())

meta_ui_1 = meta_app[meta_app.isin(free_app)]
meta_ui_2 = free_app[free_app.isin(meta_app)]
assert len(meta_ui_1) == len(meta_ui_2), 'wrong'
meta_free_ui = meta_data[meta_data['PACKAGE_NAME'].isin(meta_ui_1)]
if not os.path.exists(path('meta_free_UI')):
    meta_free_ui.to_csv(path('meta_free_UI'))
else:
    meta_free_ui = pd.read_csv(path('meta_free_UI'))

meta_ui_3 = meta_app[meta_app.isin(non_free_app)]
meta_ui_4 = non_free_app[non_free_app.isin(meta_app)]
assert len(meta_ui_3) == len(meta_ui_4), 'wrong'
meta_non_free_ui = meta_data[meta_data['PACKAGE_NAME'].isin(meta_ui_3)]
if not os.path.exists(path('meta_non_free_UI')):
    meta_non_free_ui.to_csv(path('meta_non_free_UI'))
else:
    meta_non_free_ui = pd.read_csv(path('meta_non_free_UI'))

## 筛选UI的代码
如果没有筛选UI，就运行代码。如果已经过滤了，就直接读取。

In [23]:
# Use keywords to get all ui related data.
# if the data has been filtered, just read them from the disk.
# Input of this cell: free; non_free.
# Output of this cell, free_ui, meta_free_ui; non_free_ui, meta_non_free_ui.


def filter_UI(data, meta_data):
    review1 = data['REVIEW_TEXT'][:len(data) // 2].str.lower()
    review2 = data['REVIEW_TEXT'][len(data) // 2:].str.lower()
    review = review1.append(review2)
    del(review1)
    del(review2)
    condition_ui = 'ux | ux|ui | ui|ugly|layout|design|gui | gui|aesthetic|interface|hideous|ui/ux|appearance|color|colour|bright|dark|screen|logo|icon|photo|picture|image|easy to use|user friendly|tacky|user interface|user unfriendly|un-friendly|user experience|simple to use|navigation|bland|menu|easy to operate|pic | pic|usability|simple interface|homescreen|simple interface|easy to operate|easy naviagtion|easy to handle|dpi |dpi |webpage|resolution|hard to use|bloat'
    # there are some reviews not in the metadata, so we get otherrid of them, and this will not impact the whole conclusion

    data = data[data['PACKAGE_NAME'].isin(meta_data['PACKAGE_NAME'])]
    print('use keywords to filter ui')
    filter_ui = review.str.contains(condition_ui, na=False)
    ui_data = data[filter_ui]
    # all_ui = free_ui.append(non_free_ui)
    return ui_data

# if the data exists, skip it and just read it.
# for free
if not os.path.exists(path('free_UI')):
    free_ui = filter_UI(free, meta_data)
    print('number of selected free ui reviews: %s' % len(free_ui))
    print('wirte the data: please wait')
    free_ui.to_csv(path('free_UI'))
else:
    print('read free UI data from the disk')
    free_ui = pd.read_csv(path('free_UI'))
    print('Number of free UI review: %s' % len(free_ui))
    
# for non-free
if not os.path.exists(path('non_free_UI')):
    non_free_ui = filter_UI(non_free, meta_data)
    print('number of selected non_free ui reviews: %s' % len(non_free_ui))
    print('congratulation! data process is completed!')
    non_free_ui.to_csv(path('non_free_UI'))
else:
    print('read non_free UI datafrom the disk')
    non_free_ui = pd.read_csv(path('non_free_UI'))
    print('Number of non-free UI review: %s' % len(non_free_ui))

read free UI data from the disk
Number of free UI review: 3035518
read non_free UI datafrom the disk
Number of non-free UI review: 319627


 ### 对原数据和对过滤后的数据的统计，包括app的数量和review的数量
 Statistics of raw data and filtered data.

In [7]:
# data for table1 (i.e., the statistics of app)
print('number of apps in meta data %s' % len(meta_data['PACKAGE_NAME']))
print('number of categories in meta data %s' % len(meta_data['APP_SUB_CATEGORY'].unique()))
print('number of free apps in meta data %s' % len(meta_data['PACKAGE_NAME'][meta_data['IS_2018_TOP_FREE_APPS'] == 'Y']))
print('number of free categories in meta data %s' % len(meta_data['APP_SUB_CATEGORY'][meta_data['IS_2018_TOP_FREE_APPS'] == 'Y'].unique()))
print('number of non free apps in meta data %s' % len(meta_data['PACKAGE_NAME'][meta_data['IS_2018_TOP_NON_FREE_APPS'] == 'Y']))
print('number of non free categories in meta data %s' % len(meta_data['APP_SUB_CATEGORY'][meta_data['IS_2018_TOP_NON_FREE_APPS'] == 'Y'].unique()))


NameError: name 'meta_data' is not defined

### 统计总分 & 统计UI的总分

In [ ]:
# #### Statistics of UI rating and all rating
# #### 有两种总分，挑一种高的来
all_ui = free_ui.append(non_free_ui)
# all_data = free.append(non_free)
meta_ui = meta_free_ui.append(meta_non_free_ui)
print(all_ui.describe())
# all_data.describe()
meta_ui['AGGREGATED_RATING'].mean()

# 统计reply的代码

## 数据量太大，用迭代的方法读取，分块处理，因为只需要获得有update的，所以用pandas的duplicated()这个方法

In [95]:
# revisions = pd.read_csv(path('reply_data'))[['REVIEW_ID', 'RATING', 'REVIEW_DATE', 'REVIEW_TEXT', 'REPLY_TEXT', 'REPLY_DATE']]
reader = pd.read_csv(path('reply_data'), iterator=True)

In [97]:
import pandas as pd
import numpy as np
from tqdm import tqdm,tqdm_notebook
 

# @execution_time
def reader_pandas(reader, chunkSize=100000, patitions=100):
#     reader = pd.read_csv(file, iterator=True)
    chunks = []
    with tqdm(range(patitions), 'Reading ...') as t:
        for _ in t:
            try:
                chunk = reader.get_chunk(chunkSize)
                chunks.append(chunk)
            except StopIteration:
                break
    if chunks:
        return pd.concat(chunks, ignore_index=True)
    else:
        print('Reading finished')
        return None
# big_dup = []
# with tqdm(range(10), 'Writing...') as t:
#    for i in t:
if True:
    with tqdm_notebook(range(10), 'Handling...') as t:
        duplicate = []
        for i in t:
            revisions = reader_pandas(reader)
            if revisions is not None:
                dup_filter = revisions['REVIEW_ID'].duplicated(keep=False)
                reply_filter = pd.isnull(revisions['REPLY_TEXT']) == False
                dup = revisions[dup_filter | reply_filter]
                duplicate.append(dup)
            else:
                break
        pd_duplicate = pd.concat(duplicate, ignore_index=True)
        #pd_duplicate.to_csv('dup_%s.csv' % str(i))
        #big_dup.append(pd_duplicate)

Reading ...:   0%|                                                                             | 0/100 [00:00<?, ?it/s]


Reading finished



### 数据读取完毕，开始查看

In [99]:
pd_duplicate = pd.concat(duplicate, ignore_index=True)
pd_duplicate

,REVIEW_REVISION_HISTORY_ID,REVIEW_ID,APP_ID,PACKAGE_NAME,RATING,REVIEW_DATE,REVIEW_TITLE,REVIEW_TEXT,REPLY_TEXT,REPLY_DATE,REVIEWER_NAME,REVISION_TIMESTAMP
0,122920440,107603279,40951,com.teamviewer.host.market,2.0,2015-11-09,NaN,Never actually shows my screen. But i can see ...,Thank you for using the app. To analyze this i...,2015-11-10,Red Baran,2018-10-17
1,122920553,107603391,40931,com.teatimemedia.mashlite,5.0,2017-04-10,NaN,This was the most popular games in my school ...,NaN,NaN,Roses of The Darkness,2018-10-17
2,122920561,107603394,40951,com.teamviewer.host.market,2.0,2016-10-23,No support for Motorola,Title says it all... I downloaded in hopes of ...,We are happy to announce that it is possible t...,2017-09-08,A_D3MON,2018-10-17
3,122920617,107603451,40942,com.teazel.crossword.us.full,5.0,2017-01-21,NaN,Great app and I'm not forced to go through ad ...,NaN,NaN,A Google user,2018-10-17
4,122920622,107603456,40936,com.teamsnap.teamsnap,3.0,2018-06-01,NaN,The app was having issues I reach out to suppo...,Update! We identified a bug and it has been fi...,2018-05-31,Erick Siegmund,2018-10-17
5,122920774,107603610,40951,com.teamviewer.host.market,1.0,2016-01-10,P8 lite add-on is missing,Resolve the issue or tell me the solution.,We are happy to announce that it is possible t...,2017-09-08,Rama Jani,2018-10-17
6,122920911,107603749,40936,com.teamsnap.teamsnap,1.0,2018-07-05,NaN,I like the app but for the last year I have no...,Hi Joshua if you update to the latest version ...,2018-08-13,Joshua R,2018-10-17
7,122920948,107603782,40951,com.teamviewer.host.market,1.0,2018-04-02,NaN,Host crashes all the time,Hello Thank you for your patience. The proble...,2018-04-13,soul string,2018-10-17
8,122920953,107603790,40961,com.tecarta.NIV50thBible,3.0,2018-06-28,NaN,Don't like it much. Probably going to delete.,Hi Charles sorry you're not satisfied. :( We w...,2018-06-28,Charles Moore,2018-10-17
9,122921025,107603857,40957,com.teazel,1.0,2015-06-01,It USED to be great,This program was exactly what I wanted in a di...,Hello trippyhare We goofed - our apologies. Th...,2015-06-01,Resourceful Man is Resourceful,2018-10-17


## 将非需要研究的app的数据给筛选出去

In [103]:
# 获得处理好的meta_ui
meta_free_ui = pd.read_csv(path('meta_free_ui'))
meta_non_free_ui = pd.read_csv(path('meta_non_free_ui'))
meta_ui = meta_free_ui.append(meta_non_free_ui)
meta_ui['APP_ID']

0       10670
1       10671
2       10672
3       10675
4       10678
5       10682
6       10686
7       10690
8       10694
9       10701
10      10705
11      10709
12      10710
13      10713
14      10714
15      10718
16      10721
17      10722
18      10725
19      10729
20      10732
21      10734
22      10736
23      10737
24      10739
25      10740
26      10741
27      10742
28      10743
29      10746
        ...  
6456    57756
6457    57779
6458    57780
6459    57783
6460    57786
6461    57793
6462    57795
6463    57798
6464    57803
6465    57805
6466    57813
6467    57815
6468    57821
6469    57874
6470    57880
6471    57882
6472    57886
6473    57888
6474    57892
6475    57903
6476    57904
6477    57923
6478    57926
6479    57941
6480    57952
6481    57955
6482    57971
6483    57996
6484    58000
6485    58007
Name: APP_ID, Length: 25029, dtype: int64

In [110]:
# 获得需要研究的revision数据，其他的则被筛选出去
ui_app_duplicate = pd_duplicate[pd_duplicate['APP_ID'].isin(meta_ui['APP_ID'])]
ui_app_duplicate

,REVIEW_REVISION_HISTORY_ID,REVIEW_ID,APP_ID,PACKAGE_NAME,RATING,REVIEW_DATE,REVIEW_TITLE,REVIEW_TEXT,REPLY_TEXT,REPLY_DATE,REVIEWER_NAME,REVISION_TIMESTAMP
0,122920440,107603279,40951,com.teamviewer.host.market,2.0,2015-11-09,NaN,Never actually shows my screen. But i can see ...,Thank you for using the app. To analyze this i...,2015-11-10,Red Baran,2018-10-17
2,122920561,107603394,40951,com.teamviewer.host.market,2.0,2016-10-23,No support for Motorola,Title says it all... I downloaded in hopes of ...,We are happy to announce that it is possible t...,2017-09-08,A_D3MON,2018-10-17
4,122920622,107603456,40936,com.teamsnap.teamsnap,3.0,2018-06-01,NaN,The app was having issues I reach out to suppo...,Update! We identified a bug and it has been fi...,2018-05-31,Erick Siegmund,2018-10-17
5,122920774,107603610,40951,com.teamviewer.host.market,1.0,2016-01-10,P8 lite add-on is missing,Resolve the issue or tell me the solution.,We are happy to announce that it is possible t...,2017-09-08,Rama Jani,2018-10-17
6,122920911,107603749,40936,com.teamsnap.teamsnap,1.0,2018-07-05,NaN,I like the app but for the last year I have no...,Hi Joshua if you update to the latest version ...,2018-08-13,Joshua R,2018-10-17
7,122920948,107603782,40951,com.teamviewer.host.market,1.0,2018-04-02,NaN,Host crashes all the time,Hello Thank you for your patience. The proble...,2018-04-13,soul string,2018-10-17
8,122920953,107603790,40961,com.tecarta.NIV50thBible,3.0,2018-06-28,NaN,Don't like it much. Probably going to delete.,Hi Charles sorry you're not satisfied. :( We w...,2018-06-28,Charles Moore,2018-10-17
10,122921155,107603991,40971,com.tecarta.amp,1.0,2014-04-30,NaN,I paid for this app and loved it but after upd...,Hi Shanon sorry you're experiencing problems. ...,2016-05-20,Shanon Plunkett,2018-10-17
11,122921194,107604021,40936,com.teamsnap.teamsnap,2.0,2018-09-09,NaN,Too buggy to feel like a proper release produc...,Sorry to hear about your recent TeamSnap exper...,2018-10-16,A Google user,2018-10-17
13,122921378,107604210,40969,com.tecarta.NASBStrongs,1.0,2017-03-23,NaN,Purchased and now its tellimg me to purchase a...,Hi Jeannette sorry you're experiencing problem...,2017-03-23,Janette W,2018-10-17


In [114]:
# 用UI关键词继续筛选
condition_ui = 'ux | ux|ui | ui|ugly|layout|design|gui | gui|aesthetic|interface|hideous|ui/ux|appearance|color|colour|bright|dark|screen|logo|icon|photo|picture|image|easy to use|user friendly|tacky|user interface|user unfriendly|un-friendly|user experience|simple to use|navigation|bland|menu|easy to operate|pic | pic|usability|simple interface|homescreen|simple interface|easy to operate|easy naviagtion|easy to handle|dpi |dpi |webpage|resolution|hard to use|bloat'
reviews = ui_app_duplicate['REVIEW_TEXT'].str.lower()
filter_ui = reviews.str.contains(condition_ui, na=False)
ui_duplicate = ui_app_duplicate[filter_ui]

In [ ]:
# 继续筛选完全没有回复的


## 进行统计和处理

In [115]:
# 查看数据的cell
ui_duplicate

,REVIEW_REVISION_HISTORY_ID,REVIEW_ID,APP_ID,PACKAGE_NAME,RATING,REVIEW_DATE,REVIEW_TITLE,REVIEW_TEXT,REPLY_TEXT,REPLY_DATE,REVIEWER_NAME,REVISION_TIMESTAMP
0,122920440,107603279,40951,com.teamviewer.host.market,2.0,2015-11-09,NaN,Never actually shows my screen. But i can see ...,Thank you for using the app. To analyze this i...,2015-11-10,Red Baran,2018-10-17
22,122921696,107604529,40947,com.teazel.crossword.full,2.0,2016-12-25,Too easy other than not enough word intersecti...,I don't think any application that I paid for ...,Hello Sorry to hear you're not happy. We have ...,2016-12-26,A Google user,2018-10-17
28,122921981,107604815,40936,com.teamsnap.teamsnap,1.0,2018-09-15,NaN,This app still has one of the worst user inter...,A search for teams feature does not exist for ...,2018-09-27,D Zar,2018-10-17
30,122922180,107605010,40936,com.teamsnap.teamsnap,3.0,2018-09-15,NaN,I have a really hard time doing things with sc...,Thank you for your feedback. We are always wor...,2018-09-26,A Google user,2018-10-17
33,122922308,107605143,40951,com.teamviewer.host.market,2.0,2017-07-04,NaN,Now fails on Huawei P9/P9+ devices since lates...,We are happy to announce that it is possible t...,2017-09-08,Nicholas Windley,2018-10-17
34,122922309,107605144,40938,com.teamviewer.teamviewer.market.mobile,2.0,2018-08-22,NaN,Last update ruined it. Was perfect. Now keyboa...,NaN,NaN,Coty Spraggs,2018-10-17
42,122922801,107605633,40951,com.teamviewer.host.market,1.0,2017-08-13,NaN,I can't interact with the host through the tea...,We are happy to announce that it is possible t...,2017-09-08,Alan Rangel,2018-10-17
47,122923040,107605875,40939,com.teamspeak.ts3client,5.0,2015-09-01,luv it,but could use an UI upgrade to material design...,NaN,NaN,Gregor Heggerlsen,2018-10-17
51,122923170,107605998,40951,com.teamviewer.host.market,1.0,2016-06-19,Not working for Moto X Play,It is only screen sharing..not remote access.p...,We are happy to announce that it is possible t...,2017-09-08,Manas Mund,2018-10-17
56,122923208,107606048,40974,com.tecarta.lasb,3.0,2015-02-07,NaN,Something seems to have changed in the last wh...,Hi Ross sorry you're experiencing problems. :(...,2015-07-16,saucerutherland,2018-10-17


In [131]:
counts = ui_duplicate['REVIEW_ID'].value_counts()
# 将series 变成一个dataframe，但是columns有问题，所以重新设置
counts = counts.reset_index()
counts['COUNTS'] = counts['REVIEW_ID']
counts['REVIEW_ID'] = counts['index']
counts = counts.drop(columns = ['index'])
counts

,REVIEW_ID,COUNTS
0,107931288,82
1,107930333,80
2,107935340,80
3,107936227,80
4,107933752,80
5,99899937,23
6,99900919,23
7,99900485,23
8,97751883,23
9,99900052,23


### 在有update的数据里面，有的时候是没有回复的，所以需要把这些给删掉

In [142]:
reply_list = []
for review_id in tqdm_notebook(counts['REVIEW_ID']):
    condition = ui_duplicate['REVIEW_ID'] == review_id
    this_revision = ui_duplicate[condition]
    if not this_revision['REPLY_TEXT'].isna().all():
        reply_list.append(this_revision)
ui_reply_revision = pd.concat(reply_list)

In [143]:
# 查看数据的cell
ui_reply_revision

,REVIEW_REVISION_HISTORY_ID,REVIEW_ID,APP_ID,PACKAGE_NAME,RATING,REVIEW_DATE,REVIEW_TITLE,REVIEW_TEXT,REPLY_TEXT,REPLY_DATE,REVIEWER_NAME,REVISION_TIMESTAMP
29262,123248455,107931288,41849,com.toastmobile.mafia,5.0,2018-02-15,NaN,This game is great but for some reason i cant ...,Sorry that you are having trouble playing our ...,2018-04-13,Curtis Chanakira,2018-10-17
628852,129749045,107931288,41849,com.toastmobile.mafia,5.0,2018-02-15,NaN,This game is great but for some reason i cant ...,Sorry that you are having trouble playing our ...,2018-04-13,Curtis Chanakira,2018-10-22
723826,130593577,107931288,41849,com.toastmobile.mafia,5.0,2018-02-15,NaN,This game is great but for some reason i cant ...,Sorry that you are having trouble playing our ...,2018-04-13,Curtis Chanakira,2018-10-24
878587,131925243,107931288,41849,com.toastmobile.mafia,5.0,2018-02-15,NaN,This game is great but for some reason i cant ...,Sorry that you are having trouble playing our ...,2018-04-13,Curtis Chanakira,2018-10-29
933843,132387994,107931288,41849,com.toastmobile.mafia,5.0,2018-02-15,NaN,This game is great but for some reason i cant ...,Sorry that you are having trouble playing our ...,2018-04-13,Curtis Chanakira,2018-10-31
966048,132682093,107931288,41849,com.toastmobile.mafia,5.0,2018-02-15,NaN,This game is great but for some reason i cant ...,Sorry that you are having trouble playing our ...,2018-04-13,Curtis Chanakira,2018-11-01
993473,132979908,107931288,41849,com.toastmobile.mafia,5.0,2018-02-15,NaN,This game is great but for some reason i cant ...,Sorry that you are having trouble playing our ...,2018-04-13,Curtis Chanakira,2018-11-01
1021121,133267879,107931288,41849,com.toastmobile.mafia,5.0,2018-02-15,NaN,This game is great but for some reason i cant ...,Sorry that you are having trouble playing our ...,2018-04-13,Curtis Chanakira,2018-11-02
1046080,133489290,107931288,41849,com.toastmobile.mafia,5.0,2018-02-15,NaN,This game is great but for some reason i cant ...,Sorry that you are having trouble playing our ...,2018-04-13,Curtis Chanakira,2018-11-03
1092642,133947114,107931288,41849,com.toastmobile.mafia,5.0,2018-02-15,NaN,This game is great but for some reason i cant ...,Sorry that you are having trouble playing our ...,2018-04-13,Curtis Chanakira,2018-11-04


In [170]:
counts = ui_reply_revision['REVIEW_ID'].value_counts()
# 将series 变成一个dataframe，但是columns有问题，所以重新设置
counts = counts.reset_index()
counts['COUNTS'] = counts['REVIEW_ID']
counts['REVIEW_ID'] = counts['index']
counts = counts.drop(columns = ['index'])
counts

,REVIEW_ID,COUNTS
0,107931288,82
1,97751883,23
2,111378739,16
3,114279409,13
4,118307288,12
5,130806951,12
6,130806956,12
7,130806948,12
8,107165700,11
9,105537488,11


#### 查看某些例子的查询代码（以APP_ID进行查询）

In [189]:
condition = ui_duplicate['REVIEW_ID'] == 103501299
ui_duplicate[condition]

,REVIEW_REVISION_HISTORY_ID,REVIEW_ID,APP_ID,PACKAGE_NAME,RATING,REVIEW_DATE,REVIEW_TITLE,REVIEW_TEXT,REPLY_TEXT,REPLY_DATE,REVIEWER_NAME,REVISION_TIMESTAMP
618237,129632302,103501299,30695,com.luutinhit.ioslauncher,5.0,2018-10-19,NaN,I give this 5 stars. This launcher is exactly ...,Thank you so much for your review. If you like...,2018-10-14,Rommel,2018-10-21
994983,132995930,103501299,30695,com.luutinhit.ioslauncher,5.0,2018-11-01,NaN,I give this 5 stars. This launcher is exactly ...,Thank you so much for your review. If you like...,2018-10-14,Rommel,2018-11-01
2340789,144956353,103501299,30695,com.luutinhit.ioslauncher,5.0,2018-12-22,NaN,I give this 5 stars. This launcher is exactly ...,Thank you so much for your review. If you like...,2018-10-14,Rommel,2018-12-23
2773912,147863750,103501299,30695,com.luutinhit.ioslauncher,5.0,2019-01-07,NaN,I give this 5 stars. This launcher is exactly ...,Thank you so much for your review. If you like...,2018-10-14,Rommel,2019-01-09
2890093,148606727,103501299,30695,com.luutinhit.ioslauncher,5.0,2019-01-11,NaN,I give this 5 stars. This launcher is exactly ...,Thank you so much for your review. If you like...,2018-10-14,Rommel,2019-01-14
3082325,150115728,103501299,30695,com.luutinhit.ioslauncher,5.0,2019-01-21,NaN,I give this 5 stars. This launcher is exactly ...,Thank you so much for your review. If you like...,2018-10-14,Rommel,2019-01-22
4862624,179020925,103501299,30695,com.luutinhit.ioslauncher,3.0,2019-01-28,NaN,I give this 5 stars. This launcher is exactly ...,Thank you so much for your review. If you like...,2018-10-14,Rommel,2019-01-30
6014295,195967810,103501299,30695,com.luutinhit.ioslauncher,2.0,2019-02-06,NaN,I give this 5 stars. This launcher is exactly ...,Thank you so much for your review. If you like...,2018-10-14,Rommel,2019-02-09
6629748,203765326,103501299,30695,com.luutinhit.ioslauncher,1.0,2019-02-16,NaN,I give this 5 stars. This launcher is exactly ...,Thank you so much for your review. If you like...,2018-10-14,Rommel,2019-02-24


### 对于对话次数的统计结果

In [178]:
result = counts['COUNTS'].value_counts()
# dic_count = {'dialogue_iteration':result.index, 'review_number':result.values}
# pd.DataFrame(dic_count)
result.sort_index()

1     275391
2      31134
3       2845
4        487
5         86
6         37
7          8
8          8
9          5
10         2
11         5
12         4
13         1
16         1
23         1
82         1
Name: COUNTS, dtype: int64

### 继续分析回复前和回复后的比分变化，和回复时间

因为前期已经全部筛选好了，所以只要以更新时间作为索引，然后获得最开始的比分和最终的比分即可，甚至可以看一下中间的比分的波动。
```pd.DateFrame(data, index=date)
```

#### 比分变化

In [191]:
date_time = ui_reply_revision.set_index(['REVIEW_DATE'])

In [ ]:
from collections import defaultdict
rating_dic = defaultdict(list)
for review_id in tqdm_notebook(counts['REVIEW_ID']):
    condition = ui_duplicate['REVIEW_ID'] == review_id
    this_dialogue = ui_duplicate[condition]
    dialogue_date = this_dialogue.set_index(['REVIEW_DATE'])
    # 评分变化的序列
    rating_list = dialogue_date.sort_index()['RATING']
    rating_dic['REVIEW_ID'].append(review_id)
    rating_dic['new_rating'].append(rating_list.iloc[-1])
    rating_dic['old_rating'].append(rating_list.iloc[0])
    rating_change = rating_list.iloc[-1] - rating_list.iloc[0]
    if rating_change > 0:
        rating_update = 'up'
    elif rating_change == 0:
        rating_update = 'same'
    elif rating_change < 0:
        rating_update = 'down'
    rating_dic['rating_update'].append(rating_update)

rating_info = pd.DataFrame(rating_dic)
rating_info

#### 回复时间

In [ ]:
for review_id in tqdm_notebook(counts['REVIEW_ID']):
    condition = ui_duplicate['REVIEW_ID'] == review_id
    this_dialogue = ui_duplicate[condition]